<!-- Autoheader begin -->
<hr/>
<div id="navtitle_3_4_jl" style="text-align:center; font-size:16px">III.4 Entangling Quantum Gates for Coupled Transmon Qubits</div>
<hr/>
<table style="width: 100%">
  <tr>
    <th rowspan="2" style="width:33%; text-align:center; font-size:16px">
        <a href="jl_exercise_3_3_chiral.ipynb">$\leftarrow$ previous notebook </a><br>
        <a href="jl_exercise_3_3_chiral.ipynb" style="font-size:13px">III.3 Using Krotov's method to separate chiral molecules</a>
    </th>
    <td style="width:33%; text-align:center; font-size:16px">
        <a href="jl_exercise_3_1_TLS.ipynb">$\uparrow$ previous part $\uparrow$</a><br>
        <a href="jl_exercise_3_1_TLS.ipynb" style="font-size:13px">III.1 Population Inversion in a Two-Level-System using Krotov's Method and GRAPE</a>
    </td>
    <th rowspan="2" style="width:33%; text-align:center; font-size:16px">
    </th>
  </tr>
  <tr style="width: 100%">
    <td style="width:33%; text-align:center; font-size:16px">
    </td>
  </tr>
</table>

<div style="text-align: right;font-size: 16px"><a href="../Python/py_exercise_3_4_gate.ipynb">👉 Python version</a></div>

---
<!-- Autoheader end -->

# Entangling Quantum Gates for Coupled Transmon Qubits

$
\newcommand{tr}[0]{\operatorname{tr}}
\newcommand{diag}[0]{\operatorname{diag}}
\newcommand{abs}[0]{\operatorname{abs}}
\newcommand{pop}[0]{\operatorname{pop}}
\newcommand{aux}[0]{\text{aux}}
\newcommand{opt}[0]{\text{opt}}
\newcommand{tgt}[0]{\text{tgt}}
\newcommand{init}[0]{\text{init}}
\newcommand{lab}[0]{\text{lab}}
\newcommand{rwa}[0]{\text{rwa}}
\newcommand{bra}[1]{\langle#1\vert}
\newcommand{ket}[1]{\vert#1\rangle}
\newcommand{braket}[1]{\langle#1\rangle}
\newcommand{Bra}[1]{\left\langle#1\right\vert}
\newcommand{Ket}[1]{\left\vert#1\right\rangle}
\newcommand{Braket}[2]{\left\langle #1\vphantom{#2}\mid{#2}\vphantom{#1}\right\rangle}
\newcommand{op}[1]{\hat{#1}}
\newcommand{Op}[1]{\hat{#1}}
\newcommand{dd}[0]{\,\text{d}}
\newcommand{Liouville}[0]{\mathcal{L}}
\newcommand{DynMap}[0]{\mathcal{E}}
\newcommand{identity}[0]{\mathbf{1}}
\newcommand{Norm}[1]{\lVert#1\rVert}
\newcommand{Abs}[1]{\left\vert#1\right\vert}
\newcommand{avg}[1]{\langle#1\rangle}
\newcommand{Avg}[1]{\left\langle#1\right\rangle}
\newcommand{AbsSq}[1]{\left\vert#1\right\vert^2}
\newcommand{Re}[0]{\operatorname{Re}}
\newcommand{Im}[0]{\operatorname{Im}}
$

In this notebook, we'll use the `QuantumControl` framework for an optimization towards a perfectly entangling
two-qubit gate for a system of two transmon qubits with a shared transmission
line.

In [ ]:
using QuantumControl

It goes through three progressively more advanced optimizations:

1. The direct optimization for a $\Op{O} = \sqrt{\text{iSWAP}}$ gate with a
   standard square-modulus functional
2. The direct maximization of of the gate concurrence with semi-automatic differentiation
3. The optimization towards a perfect entangler using the functional demonstrated in [Watts *et al.*, Phys. Rev. A 91, 062306 (2015)](https://michaelgoerz.net/#WattsPRA2015) and [Goerz *et al.*, Phys. Rev. A 91, 062307 (2015)](https://michaelgoerz.net/#GoerzPRA2015)


While the first example evaluates the gradient of the optimization functional
analytically, the latter two are examples for the use of automatic
differentiation, or more specifically semi-automatic differentiation, as
developed in [Goerz *et al.*, Quantum 6, 871 (2022)](https://quantum-journal.org/papers/q-2022-12-07-871/). The optimization of the gate
concurrence specifically illustrates the optimization of a functional that is
inherently non-analytical.

This notebook builds upon the simpler applications of Krotov's method in [Exercise III.2](jl_exercise_3_2_lambda.ipynb) and [Exercise III.3](jl_exercise_3_3_chiral.ipynb). In those examples, the optimization had to consider only the time evolution of a single quantum state. In contrast, for the optimization of a quantum gate, the optimization functional takes into account the evolution of multiple quantum states (the logical two-qubit basis states $\ket{00}$, $\ket{01}$, $\ket{10}$, $\ket{11}$). You will learn in this exercise how to work with this more involved functionals, and how the functional and the dynamics affect the optimization in Krotov's method.

This notebook serves as a nice illustration of using optimal control in a quantum information context, extending the simpler optimizations discussed previously for a two-level-system (i.e. a qubit) in [Exercise II.1](py_exercise_2_1_TLS.ipynb) and [Exercise III.1](py_exercise_3_1_TLS.ipynb).

Note that this exercise goes significantly beyond its [Python counterpart](../Python/py_exercise_3_4_gate.ipynb), in using a more realistic model where the logical subspace is embedded in a larger Hilbert space. It also uniquely illustrates the use of semi-automatic differentiation with both Krotov's method and GRAPE.

## Setup

Krotov's Method and GRAPE can be used interchangeably in these optimizations:

In [ ]:
using Krotov
using GRAPE

The simulation of the dynamics will be performed via the `QuantumPropagators` package. Both the Krotov and GRAPE method work on control fields that are piecewise-constant (defined as constant for each time step within the time grid). That is, for each time step, the time evolution operator is simply $\exp(-i \Op{H}_n dt)$ with a constant $\Op{H}_n$ for the time step $dt$. For very small systems, one can evaluate the time evolution operator with direct matrix exponentiation (`QuantumPropagators.ExpProp`, as we have used in previous examples). For slightly larger system, a very efficient method for simulating the time evolution in each time step is to expand the matrix exponential into Chebychev polynomials. This is implemented as the `Cheby` propagation method:

In [ ]:
using QuantumPropagators: Cheby

For visualization, we will use the `Plots` package

In [ ]:
using Plots

# Set up thicker default lines in plots
Plots.default(
    linewidth               = 2.0,
    foreground_color_legend = nothing,
    background_color_legend = RGBA(1, 1, 1, 0.8)
)

In [ ]:
# Some utilities for showing hints and solutions
include(joinpath("utils", "exercise_3_gate.jl"));
include(joinpath("utils", "show_code.jl"));  # @show_code macro

## Model

<img src="../figures/transmon.png" alt="Transmon System" style="width: 1200px;"/>

In this notebook we consider a system of two [transmon](https://en.wikipedia.org/wiki/Transmon) qubits (the green objects in the illustration above) with a shared transmission line ("cavity", shown above in blue), as described in [Blais *et al.*, Phys. Rev. A 75, 032329 (2007)](https://arxiv.org/abs/cond-mat/0612038). Each qubit is an anharmonic (a so-called Duffing) oscillator

$$
\Op{H}_q = \left(\omega_q - \frac{\alpha_q}{2}\right) \Op{n} + \frac{\alpha_q}{2} \Op{n}^2
$$

where $\omega_q$ is the separation of the two lowest "qubit" levels $\ket{0}$ and $\ket{1}$, $\alpha_q$ is the qubit anharmonicity, and $\Op{n}$ is the number operator. The two qubits can exchange excitations with the cavity. In the so-called dispersive limit the coupling $g$ between each qubit and the cavity is weak, such that the cavity only has "virtual excitations" which can be eliminated to obtain a simpler "effective" model containing only transmon levels. In this effective model, the two qubits have an always-on coupling $J$, i.e., they exchange excitations via the cavity at a certain rate. In addition, the microwave driving field in the transmission line drives the transitions between the transmon levels with an effective coupling $g^\text{eff}$. Since we want to obtain a proper qubit, the population should only be in the two lowest levels $\ket{0}$ and $\ket{1}$ - the so-called logical levels. However, during gate operation, population may leak into higher levels of each transmon, depending on the strength of the driving field and the magnitude of the anharmonicity. This is not an issue as long as the population returns to the logical subspace at the end of the gate operation. Else the population in higher levels would need to be considered as "lost", reducing the gate fidelity. Numerically, we can truncated the transmon to keep the numerical effort manageable. For example, if we can verify that the control field only populates level $\ket{0}$ – $\ket{4}$ at any time it is appropriate to only consider $N = 6$ levels in our model.

We will write the Hamiltonian in units of GHz, where a qubit frequency of "4.3 GHz" means that the qubit levels are separated by an energy of $E = h \cdot 4.3\text{GHz} = \hbar \cdot 2\pi \cdot 4.3\text{GHz}$. When solving the Schrödinger equation numerically, we always work with $\hbar = 1$, which motivates the implicit factor of $2\pi$. Accordingly, the natural unit of time in this unit system are nanoseconds.

In [ ]:
const GHz = 2π
const MHz = 0.001GHz
const ns = 1.0
const μs = 1000ns;

The Hamiltonian and parameters are taken from
Ref. [Goerz *et al.*, Phys. Rev. A 91, 062307
   (2015), Table 1](https://michaelgoerz.net/#GoerzPRA2015). We can write out the effective two-transmon Hamiltonian as

In [ ]:
⊗ = kron
const 𝕚 = 1im
const N = 6  # levels per transmon

using LinearAlgebra
using SparseArrays


function transmon_hamiltonian(;
    Ωre,
    Ωim,
    N=N,  # levels per transmon
    ω₁=4.380GHz,
    ω₂=4.614GHz,
    ωd=4.498GHz, # frequency of rotating frame
    α₁=-210MHz,
    α₂=-215MHz,
    J=-3MHz,
    λ=1.03,
    use_sparse=:auto
)
    𝟙 = SparseMatrixCSC{ComplexF64,Int64}(sparse(I, N, N))
    b̂₁ = spdiagm(1 => complex.(sqrt.(collect(1:N-1)))) ⊗ 𝟙
    b̂₂ = 𝟙 ⊗ spdiagm(1 => complex.(sqrt.(collect(1:N-1))))
    b̂₁⁺ = sparse(b̂₁')
    b̂₂⁺ = sparse(b̂₂')
    n̂₁ = sparse(b̂₁' * b̂₁)
    n̂₂ = sparse(b̂₂' * b̂₂)
    n̂₁² = sparse(n̂₁ * n̂₁)
    n̂₂² = sparse(n̂₂ * n̂₂)
    b̂₁⁺_b̂₂ = sparse(b̂₁' * b̂₂)
    b̂₁_b̂₂⁺ = sparse(b̂₁ * b̂₂')

    ω̃₁ = ω₁ - ωd  # rotating frame frequencies
    ω̃₂ = ω₂ - ωd

    Ĥ₀ = sparse(
        (ω̃₁ - α₁ / 2) * n̂₁ +
        (α₁ / 2) * n̂₁² +
        (ω̃₂ - α₂ / 2) * n̂₂ +
        (α₂ / 2) * n̂₂² +
        J * (b̂₁⁺_b̂₂ + b̂₁_b̂₂⁺)
    )

    Ĥ₁re = (1 / 2) * (b̂₁ + b̂₁⁺ + λ * b̂₂ + λ * b̂₂⁺)
    Ĥ₁im = (𝕚 / 2) * (b̂₁⁺ - b̂₁ + λ * b̂₂⁺ - λ * b̂₂)

    if ((N < 5) && (use_sparse ≢ true)) || use_sparse ≡ false
        # For small Hilbert spaces, it it more efficient to work with
        # dense matrices
        H = hamiltonian(Array(Ĥ₀), (Array(Ĥ₁re), Ωre), (Array(Ĥ₁im), Ωim))
    else
        H = hamiltonian(Ĥ₀, (Ĥ₁re, Ωre), (Ĥ₁im, Ωim))
    end
    return H

end;

The Hamiltonian is written in a rotating frame, so in general. This implies that the control
field is allowed to be complex-valued in general. A common strategy for complex controls is to separate them into two real-valued control fields - one for the real part and one for the imaginary part. Initially, the
imaginary part is set to zero, which corresponds to a field exactly at the frequency of the rotating frame.

We choose a pulse duration of 400 ns. The guess pulse amplitude is 35 MHz,
with a 15 ns switch-on/-off time. The amplitude already contains the $g^\text{eff}$ factor, i.e., how much amplitude of the physical field the transmon actually feels. The factor might be different for the two qubits, which we've taken into account here with a factor $\lambda$.

This switch-on/-off must be maintained in
the optimization: A pulse that does not start from or end at zero would not
be physical. For GRAPE, we can achieve this by using a `ShapedAmplitude`:

In [ ]:
using QuantumControl.Amplitudes: ShapedAmplitude

This allows to have a control amplitude $Ω(t) = S(t) ϵ(t)$ where $S(t)$
is a fixed shape and $ϵ(t)$ is the pulse directly tuned by the
optimization. We start with a constant $ϵ(t)$ and do not place any
restrictions on how the optimization might update $ϵ(t)$. Krotov's method has an alternative way to ensure this boundary condition by employing an "update shape" which limits the pulse update in each iteration to be zero at the boundary points. However, note that the `ShapedAmplitude` makes this unnecessary and works equally well for GRAPE and Krotov's method.

Note that passing `tlist` to `ShapedAmplitude` discretizes both the control
and the shape function to the midpoints of the `tlist` array.

In [ ]:
using QuantumControl.Shapes: flattop

function guess_amplitudes(; T=400ns, E₀=35MHz, dt=0.1ns, t_rise=15ns)

    tlist = collect(range(0, T, step=dt))
    shape(t) = flattop(t, T=T, t_rise=t_rise)
    Ωre = ShapedAmplitude(t -> E₀, tlist; shape)
    Ωim = ShapedAmplitude(t -> 0.0, tlist; shape)

    return tlist, Ωre, Ωim

end

tlist, Ωre_guess, Ωim_guess = guess_amplitudes();

We can visualize this as follows

In [ ]:
using QuantumControl.Controls: discretize

function plot_complex_pulse(tlist, Ω; time_unit=:ns, ampl_unit=:MHz, kwargs...)

    Ω = discretize(Ω, tlist)  # make sure Ω is defined on *points* of `tlist`

    ax1 = plot(
        tlist ./ eval(time_unit),
        abs.(Ω) ./ eval(ampl_unit);
        label="|Ω|",
        xlabel="time ($time_unit)",
        ylabel="amplitude ($ampl_unit)",
        kwargs...
    )

    ax2 = plot(
        tlist ./ eval(time_unit),
        angle.(Ω) ./ π;
        label="ϕ(Ω)",
        xlabel="time ($time_unit)",
        ylabel="phase (π)"
    )

    plot(ax1, ax2, layout=(2, 1))

end

fig = plot_complex_pulse(tlist, Array(Ωre_guess) .+ 𝕚 .* Array(Ωim_guess))

We now connect the Hamiltonian with these control fields:

In [ ]:
H = transmon_hamiltonian(Ωre=Ωre_guess, Ωim=Ωim_guess);

## Logical basis for two-qubit gates

For simplicity, we define the qubits in the *bare* basis, i.e.
ignoring the static coupling $J$.

In [ ]:
function ket(i::Int64; N=N)
    Ψ = zeros(ComplexF64, N)
    Ψ[i+1] = 1
    return Ψ
end

function ket(indices::Int64...; N=N)
    Ψ = ket(indices[1]; N=N)
    for i in indices[2:end]
        Ψ = Ψ ⊗ ket(i; N=N)
    end
    return Ψ
end

function ket(label::AbstractString; N=N)
    indices = [parse(Int64, digit) for digit in label]
    return ket(indices...; N=N)
end;

In [ ]:
basis = [ket("00"), ket("01"), ket("10"), ket("11")];

In a practical context, it would be more appropriate to work in the "dressed" basis of the eigenstates closest to the bare states. However, since $J$ is relatively small, the bare and dressed states are quite close to each other, and for the purpose of this example we avoid the complication of having to diagonalize the Hamiltonian and choosing the correct basis states.

## Optimizing for a specific quantum gate

Our target gate is $\Op{O} = \sqrt{\text{iSWAP}}$:

In [ ]:
SQRTISWAP = [
    1  0    0   0
    0 1/√2 𝕚/√2 0
    0 𝕚/√2 1/√2 0
    0  0    0   1
];

While we could optimize for any other gate like the standard CNOT and CPHASE gates as well, it turns out that the interaction terms in the Hamiltonian allow for the gate $\sqrt{\text{iSWAP}}$ to arise very naturally for this system.

For each basis state, we get a target state that results from applying the
gate to the basis state (you can convince yourself that this equivalent
multiplying the transpose of the above gate matrix to the vector of basis
states):

In [ ]:
basis_tgt = transpose(SQRTISWAP) * basis;

The optimization aims to bring the trajectory of each basis
state to the corresponding target state:

In [ ]:
trajectories = [
    Trajectory(initial_state=Ψ, target_state=Ψtgt, generator=H) for
    (Ψ, Ψtgt) ∈ zip(basis, basis_tgt)
];

We can analyze how all basis states evolve under the guess controls in a single call:

In [ ]:
guess_states = propagate_trajectories(trajectories, tlist; method=Cheby, use_threads=true);

The gate implemented by the guess controls can be found as follows

In [ ]:
U_guess = [basis[i] ⋅ guess_states[j] for i = 1:4, j = 1:4];

We will optimize these trajectories with a square-modulus functional

$$
J_{T,sm}
= 1 - \Bigg\vert\frac{1}{4}\sum_{k=1}^{4} \underbrace{\langle \Psi_k(T) | \Psi_k^{\text{tgt}}\rangle}_{\equiv \tau_k}\Bigg\vert^2
= 1 - \frac{1}{16} \sum_{k,l=1}^{4} \underbrace{\langle \Psi_l^{\text{tgt}} | \Psi_l(T) \rangle}_{\equiv\tau_l^*} \; \underbrace{\langle \Psi_k(T) | \Psi_k^{\text{tgt}}\rangle}_{\equiv \tau_k}
$$

where $\ket{\Psi_k(T)}$ is the result of forward-propagating the basis state $\ket{\phi_k}$ (i.e., $\ket{\phi_1} = \ket{00}$, $\ket{\phi_2} = \ket{01}$, etc.)

In [ ]:
using QuantumControl.Functionals: J_T_sm

The initial value of the functional is

In [ ]:
J_T_sm(guess_states, trajectories)

which yields the gate error

In [ ]:
1 - (abs(tr(U_guess' * SQRTISWAP)) / 4)^2

An illustration on the way the pulse update is computed in Krotov's method can be found in part (b) of the figure below.

<img src="../figures/schemes.svg" alt="Schemes" style="width: 1200px;"/>

The figure is taken from from the paper [Goerz *et al.*, Quantum 6, 871 (2022)](https://quantum-journal.org/papers/q-2022-12-07-871/), where more details can be found.

Here, we'll focus on one essential feature: both in GRAPE and Krotov, the scheme involves a forward-propagation of the four basis states ($\ket{\phi_k} \rightarrow \ket{\Psi_k(T)}$ in the diagram), followed by a backward propagation of a set of states $\ket{\chi_k}$ with the boundary condition

$$
\ket{\chi_k(T)} = - \frac{\partial J_T}{\partial \bra{\Psi_k(T)}}
$$

This is how the functional enters the equations for the iterative optimization in Krotov's method as well as in GRAPE!

### Problem 1: boundary condition for the backward propagation

The `QuantumControl.optimize` function, respectively the `QuantumControl.ControlProblem` takes a function `chi` as a keyword argument that constructs the states $\ket{\chi_k(T)}$ from the set of forward-propagated states $\ket{\Psi_k(T)}$ and the `trajectories` (which contain the `target_state` for each basis states).

For the $J_{T,sm}$ as defined above, you will find that $\ket{\chi_k(T)}$ are proportional to the target states $\ket{\Psi_k^{\text{tgt}}}$. Calculate with pen and paper the derivative $-\partial J_T / \partial \bra{\Psi_k(T)}$ and fill in the proportionality factor $\alpha$ below:

In [ ]:
"""Set χ-states in-place.

The `!` in the function name is a Julia convention
that the function mutates its first argument.
An optimizer may call the function with keyword arguments,
which we should ignore here.
"""
function chi!(χ, Ψ, trajectories; kwargs...)
    τ = [Ψ[k] ⋅ trajectories[k].target_state for k = 1:4]
    α = # fill in proportionality factor
    for k = 1:4
        χ[k] .= α .* trajectories[k].target_state
    end
end

In [ ]:
# problem_1.hint

In [ ]:
# problem_1.solution

Now, we define the full optimization problems on top of the list of
trajectories, and with the optimization functional and the definition of `chi`:

In [ ]:
problem = ControlProblem(
    trajectories,
    tlist;
    iter_stop=100,
    J_T=J_T_sm,
    chi=chi!,
    check_convergence=res -> begin
        (
            (res.J_T > res.J_T_prev) &&
            (res.converged = true) &&
            (res.message = "Loss of monotonic convergence")
        )
        ((res.J_T <= 1e-3) && (res.converged = true) && (res.message = "J_T < 10⁻³"))
    end,
    prop_method=Cheby,
    use_threads=true,
);

Solve the optimization problem with GRAPE:

In [ ]:
opt_result = optimize(problem; method=GRAPE)

Take a look at what happens if the `chi!` function is implemented incorrectly. For example, use `τ` instead of `τ'`, or try to use the wrong sign.

Alternatively to GRAPE, you can use Krotov's method to run the same optimization. Krotov's method has a parameter `lambda_a` that determines the overall magnitude of the pulse update (larger `lambda_a` result in smaller updates)

In [ ]:
# opt_result = optimize(problem; method=Krotov, lambda_a=10.0)

We extract the optimized control field from the optimization result and plot
the resulting amplitude.

The `optimized_controls` field of the `opt_result` contains the optimized
controls $ϵ(t)$.

In [ ]:
ϵ_opt = opt_result.optimized_controls[1] + 𝕚 * opt_result.optimized_controls[2];

These must still be multiplied by the static shape $S(t)$ that we set up
for the guess amplitudes

In [ ]:
Ω_opt = ϵ_opt .* discretize(Ωre_guess.shape, tlist)

fig = plot_complex_pulse(tlist, Ω_opt)

Compare the pulses obtained with GRAPE and Krotov's method. How does the parameter `lambda_a` in Krotov's method influence the convergence and the resulting optimized fields?

Once we are happy with the optimization result, we propagate the optimized control field to analyze the resulting
quantum gate:

In [ ]:
using QuantumControl.Controls: get_controls, substitute

opt_states = propagate_trajectories(
    substitute(
        trajectories,
        IdDict(zip(get_controls(trajectories), opt_result.optimized_controls))
    ),
    tlist;
    method=Cheby,
    use_threads=true
);

The resulting gate is

In [ ]:
U_opt = [basis[i] ⋅ opt_states[j] for i = 1:4, j = 1:4];

and we can verify the resulting fidelity

In [ ]:
(abs(tr(U_opt' * SQRTISWAP)) / 4)^2

## Maximization of the gate concurrence

Building a quantum computer requires a "universal gate" set. Traditionally, this set consists of a specific two-qubit gate (often CNOT), and all single-qubit gates (under the assumption that single-qubit gates are "easy" to realize). However the universal set does not need to contain CNOT (or some other gate) *specifically*. What matters for universal quantum computing is the ability to create entanglement.

To any two-qubit gate (any 4 × 4 unitary) a so-called "gate concurrence" can be computed, which is the maximum entanglement (i.e. concurrence) of a state that can be obtained by applying the gate to some separable input state. A `concurrence` function is implemented in the `weylchamber` package – the Weyl chamber is a mathematical structure which describes and classifies two-qubit gates in terms of entangling power and equivalence with respect to single-qubit operations. You can find an illustration of the Weyl chamber below.

<img src="../figures/weylchamber.svg" alt="Weyl Chamber" style="width: 800px;"/>

In [ ]:
using TwoQubitWeylChamber: gate_concurrence

Most of the "standard" two-qubit gates like $\sqrt{\text{iSWAP}}$ and CNOT (points Q and L, respectively, in the diagram) are "perfect entanglers":

In [ ]:
gate_concurrence(SQRTISWAP)

In [ ]:
CNOT = [
    1 0 0 0
    0 1 0 0
    0 0 0 1
    0 0 1 0
]

gate_concurrence(CNOT)

The gate concurrence of the identity or any other random $SU(2) \times SU(2)$ matrix (corresponding to single-qubit gates) is zero:

In [ ]:
gate_concurrence(I(4))

In [ ]:
function random_unitary(N)
    H = rand(N, N)
    return exp(𝕚 * (H + H'))
end

U = random_unitary(2) ⊗ random_unitary(2)
round(gate_concurrence(U); digits=10)

In general, the gate concurrence of a random $4 \times 4$ unitary is a number between 0 and 1; heavily skewing towards 1. Interestingly, the majority of $4 \times 4$ unitaries are perfect entanglers!

In [ ]:
for _ = 1:10
    display(round(gate_concurrence(random_unitary(4)); digits=2))
end

This fact makes the gate concurrence an attractive optimization target: by optimizing the entangling power of the two-qubit gate without targeting a *specific* gate, we may identify the perfect entangler that is "easiest" to achieve with the given Hamiltonian.

We can also check that the guess pulse indeed does not yet implement a perfect entangler. So let's get to work!

In [ ]:
gate_concurrence(U_guess)

For the transmon system, we also have to take into account that population may be lost from the logical subspace: the `gate_concurrence` is only well-defined for a unitary $\Op{U}$, whereas a $\Op{U}$ obtained by projecting the four propagated basis states onto the logical subspace may not be unitary.

To account for this, we can define a functional for a given gate `U` that combines the gate
concurrence with a measure of unitarity, penalizing loss of population
from the logical subspace:

In [ ]:
using TwoQubitWeylChamber: unitarity

J_T_C(U) = 0.5 * (1 - gate_concurrence(U)) + 0.5 * (1 - unitarity(U));

This `unitarity` can be directly chosen as the population in the logical subspace:

In [ ]:
@show_code unitarity(U_guess)

It turns out that the guess pulse loses about 10% of population from the logical subspace, as you can see here:

In [ ]:
1 - unitarity(U_guess)

### Problem 2: Boundary condition (χ-states) for concurrence optimization

If we want to use `J_T_C` as defined above as an optimization functional for GRAPE or Krotov's method, we would have to work out the boundary condition for the backward propagation, $\ket{\chi_k(T)} = -\partial J_T / \partial \bra{\Psi_k(T)}$.

Using pen and paper, calculate the contribution to $\ket{\chi_k(T)}$ from the `unitarity` part of the functional using

$$
\ket{\Psi_k(T)} = \Op{U} \ket{\phi_k}
\quad \Leftrightarrow \quad
U_{ij} = \braket{\phi_i|\Op{U}|\phi_j} = \braket{\phi_i | \Psi_j(T)}
$$

(where the $\ket{\phi_k}$ are the logical basis states) and

$$
\tr[\Op{U}^\dagger \Op{U}] = \sum_{ij} U_{ji}^* U_{ji}
$$

In [ ]:
# problem_2.solution

In [ ]:
# problem_2.solution_code

Beyond that, we would have to look at how the gate concurrence is calculated, either by looking at the original literature, [Kraus, Cirac. Phys. Rev. A 63, 062309 (2001)](https://arxiv.org/abs/quant-ph/0011050) and [Childs *et al.* Phys. Rev. A 68, 052311 (2003)](https://arxiv.org/abs/quant-ph/0307190). Alternatively, you may directly look at the code for the functions `c1c2c3` and `concurrence`:

In [ ]:
@show_code gate_concurrence(U_guess)

As we can see, the gate concurrence is a function of the "Weyl chamber coordinates" $c_1$, $c_2$, $c_3$ (see the diagram above). These coordinates are defined as

In [ ]:
using TwoQubitWeylChamber: weyl_chamber_coordinates
@show_code weyl_chamber_coordinates(U_guess)

Once we have access to the coordinates, we can take a look at how they enter the gate concurrence:

In [ ]:
c₁, c₂, c₃ = weyl_chamber_coordinates(U_guess)
@show_code gate_concurrence(c₁, c₂, c₃)

Explain with the code snippets above why it would be difficult to calculate the derivative $\frac{\partial J_T}{\partial \bra{\Psi_k(T)}}$.

In [ ]:
# problem_3.hint

In [ ]:
# problem_3.solution

When it is not feasible to write out the derivative $\partial J_T / \partial \bra{\Psi_k(T)}$ by hand, the use of *automatic differentiation* (AD) is a possible solution. Automatic differentiation keeps track of the low-level computational steps while evaluating a function. It then uses a stupendous application of the chain rule to calculate a derivative of the function. This way, it can differentiate even through a non-analytical step like the determination of eigenvalues via `eigvals`.

Julia has several AD frameworks. One of the most established ones is [Zygote](https://fluxml.ai/Zygote.jl/stable/):

In [ ]:
using Zygote

We can make use of `Zygote` by telling the `QuantumControl` package to use it.

In [ ]:
QuantumControl.set_default_ad_framework(Zygote)

Specifically, `QuantumControl` provides a `make_gate_chi` function that takes a function of the form of `J_T_C` that we have defined above, and wraps it in `Zygote`, returning a function that can be passed to `propagate` as `chi`.

In [ ]:
using QuantumControl.Functionals: make_gate_chi

Note that `J_T_C` itself is also not quite in the form that `optimize` expects: it takes the 4 × 4 matrix `U` as an input, not the propagated basis states. The `QuantumControl` package provides `gate_functional` to automatically make the conversion $U_{ij} = \braket{\phi_i|\Op{U}|\phi_j} = \braket{\phi_i | \Psi_j(T)}$

In [ ]:
using QuantumControl.Functionals: gate_functional

Thus, we can now re-run the optimization with`J_T_C` as the functional to find an arbitrary perfect entangler, instead of the $\sqrt{\text{iSWAP}}$ gate specifically:

In [ ]:
opt_result_C = optimize(
    problem;
    method=GRAPE,
    J_T=gate_functional(J_T_C),
    chi=make_gate_chi(J_T_C, trajectories),
)

Alternatively, run this with Krotov's method

In [ ]:
#=
opt_result_C = optimize(
    problem;
    method=Krotov,
    J_T=gate_functional(J_T_C),
    chi=make_gate_chi(J_T_C, trajectories),
    lambda_a=1,
)
=#

We extract the optimized control field from the optimization result and plot
it

In [ ]:
ϵ_opt_C = opt_result_C.optimized_controls[1] + 𝕚 * opt_result_C.optimized_controls[2]
Ω_opt_C = ϵ_opt_C .* discretize(Ωre_guess.shape, tlist)

fig = plot_complex_pulse(tlist, Ω_opt_C)

We then propagate the optimized control field to analyze the resulting
quantum gate:

In [ ]:
opt_states_C = propagate_trajectories(
    substitute(
        trajectories,
        IdDict(zip(get_controls(trajectories), opt_result_C.optimized_controls))
    ),
    tlist;
    method=Cheby,
    use_threads=true
);

U_opt_C = [basis[i] ⋅ opt_states_C[j] for i = 1:4, j = 1:4];

You should now indeed find that we have achieved a perfect entangler

In [ ]:
gate_concurrence(U_opt_C)

with negligible loss of population from the logical subspace!

In [ ]:
1 - unitarity(U_opt_C)

## Optimizing for perfect entangler in the Weyl chamber

The realization that the gate concurrence is a non-analytic functional for which one cannot write down a derivative prompts the question whether one can formulate an equivalent functional that *is* analytically derivable. That approach was demonstrated in [Watts *et. al*, Phys. Rev. A 91, 062306 (2015)](https://michaelgoerz.net/#WattsPRA2015) and [Goerz *et al.*, Phys. Rev. A 91, 062307 (2015)](https://michaelgoerz.net/#GoerzPRA2015).

The basic idea is that the mathematical structure of the two-qubit gates in the Weyl chamber that we also used in the definition of the gate concurrence has a *geometric* interpretation. The set of perfect entanglers form a compact polyhedron inside the Weyl chamber (the shaded region in the diagram), and we can optimize for a perfect entangler by minimizing the geometric distance to the surface of that polyhedron.

The distance measure to the surface of the polyhedron is implemented in the `TwoQubitWeylChamber` package:

In [ ]:
using TwoQubitWeylChamber: D_PE

We convert it to a functional with `gate_functional`, adding a unitarity term, like before:

In [ ]:
J_T_PE = gate_functional(D_PE; unitarity_weight=0.5);

The geometric distance to the polyhedron of perfect
entanglers in the Weyl chamber for the guess pulse is

In [ ]:
D_PE(U_guess)

Together with the unitarity measure, this is the initial value of the
optimization functional:

In [ ]:
0.5 * D_PE(U_guess) + 0.5 * (1 - unitarity(U_guess))

In [ ]:
J_T_PE(guess_states, trajectories)

The $J_{T,PE}$ functional has an analytic derivative – it is a high-order polynomial in the states. However, that derivative is still tedious to calculate and implement. While there exists a [Python implementation](https://github.com/qucontrol/weylchamber/blob/ec96621907c74da2ed1a746292f0e64237840bec/src/weylchamber/perfect_entanglers.py#L209), it has not yet been translated to Julia. Thus, we will once again use the automatic differentiation via Zygote to obtain the $\ket{\chi_k(T)}$ states:

In [ ]:
chi_pe = make_gate_chi(D_PE, trajectories; unitarity_weight=0.5);

We then use this in the optimization:

In [ ]:
opt_result_PE = optimize(
    problem;
    method=GRAPE,
    J_T=J_T_PE,
    chi=chi_pe,
)

Alternatively, using Krotov's method:

In [ ]:
#=
opt_result_PE = optimize(
    problem;
    method=Krotov,
    J_T=J_T_PE,
    chi=chi_pe,
    lambda_a=1.0
)
=#

We extract the optimized control field from the optimization result and plot
it

In [ ]:
ϵ_opt_PE = opt_result_PE.optimized_controls[1] + 𝕚 * opt_result_PE.optimized_controls[2]
Ω_opt_PE = ϵ_opt_PE .* discretize(Ωre_guess.shape, tlist)

fig = plot_complex_pulse(tlist, Ω_opt_PE)

We then propagate the optimized control field to analyze the resulting
quantum gate:

In [ ]:
opt_states_PE = propagate_trajectories(
    substitute(
        trajectories,
        IdDict(zip(get_controls(trajectories), opt_result_PE.optimized_controls))
    ),
    tlist;
    method=Cheby,
    use_threads=true
);

U_opt_PE = [basis[i] ⋅ opt_states_PE[j] for i = 1:4, j = 1:4];

You should once again find that we have indeed achieved a perfect entangler:

In [ ]:
gate_concurrence(U_opt_PE)

Moreover, we have reduced the population loss to less than 4%

In [ ]:
1 - unitarity(U_opt_PE)

Tweak the relative weight of the `D_PE` and the unitarity term in the functional to see whether you may reduce this value even further!

## Next steps

You may find even more examples beyond the scope of this tutorial as part of the
[`QuantumControl` documentation](https://juliaquantumcontrol.github.io/QuantumControlExamples.jl/stable/). This includes an [example for optimizing a quantum gate in an open quantum system](https://juliaquantumcontrol.github.io/QuantumControlExamples.jl/stable/examples/rho_3states/#Optimization-of-a-Dissipative-Quantum-Gate). You may also find the [examples from the documentation of the Python `krotov` package](https://qucontrol.github.io/krotov/v1.2.1/09_examples.html) instructive (all of which can be relatively easily translated to Julia).

<!-- Autofooter begin -->

---

[⬆︎ jump to top](#navtitle_3_4_jl)
<!-- Autofooter end -->